# Airbnb Rio de Janeiro - Feature engineering

# 1. Introdução

Esse notebook propõe um processo de feature engineering para os [dados do Airbnb na cidade do Rio de Janeiro](https://insideairbnb.com/get-the-data/#:~:text=Rio%20de%20Janeiro%2C%20Rio%20de%20Janeiro%2C%20Brazil). 

A base de dados original foi previamente explorada e modificada durante a etapa de [análise exploratória (EDA)](https://github.com/BPRateiro/rio-airbnb-data-analysis/blob/main/notebooks/eda.ipynb). Os dados incluem informações detalhadas sobre a disponibilidade de listagens, características dos imóveis, interações de usuários, e preços. 

O objetivo principal é transformar essas informações para melhorar a capacidade preditiva de modelos de machine learning e facilitar análises mais profundas.

## 1.1 Configurações e bibliotecas

In [6]:
import pandas as pd
from summarytools import dfSummary
from sklearn.impute import KNNImputer

## 1.2 Leitura dos dados

In [7]:
df_inicial = pd.read_parquet('../data/bronze/feature_engineering_input.parquet')
df_inicial.drop(columns=['host_location'], inplace=True) # Acredito que não será uma informação relevante
dfSummary(df_inicial)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,accommodates[int64],Mean (sd) : 4.0 (2.3)min < med < max:1.0 < 4.0 < 16.0IQR (CV) : 3.0 (1.7),16 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/TGe4hAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACz0lEQVR4nO3dvY7aQBRA4TuwEPNnC4igo0nHSim25AFS7sPuIwSJIiWhTUWDkBAywTFiMKRI0q0A2xDPRefrB1wc767nasfmdDoJ4LpS0RcAXOOp6As4xxhTE5FqxuX70+kU3/J6UBxnQzXG1Lrd7qvv++0s6zebzdoY80asj8HZUEWk6vt+ezQaxUEQ7NIsDMPQm0wm7dVqVRURQn0ALocqIiJBEOx6vd6vDEtrN78YFIaHKahAqFCBUKECoUIFQoUKhAoVCBUqECpUuLjhz7wdLjgbKvN2uOLST1Tm7XDCVbN+5u0oGg9TUIFQoQKhQgVChQqEChUIFSoQKlQgVKhAqFCBUKECoUIFQoUKhAoVCBUqECpUIFSoQKhQgVChAqFCBUKFCoQKFQgVKhAqVCBUqECoUIFQoQKhQgVChQqEChUIFSrc9RWTSZJURMQ3xmRZ7h+Px8qNLwlK3S3UKIoq1trPg8GgXC6XUx0CLCJirfWSJHne7XZfRST12aw5bxIRjnV3yt1C3e/3T57n1V9eXnb9fn+ddv18Pm/PZrP64XAop12b9yYR4Vh319z97dKNRiPTadXr9TrzadV5bxKOdXeP869BzyPrTfIXx7o7hKd+qECoUIFQoQKhQgVChQqEChUIFSo89D5qHjlHsIxfb4xQ35F3BMv49fYI9R15RrCMX++DUM/IMYJl/HpjPExBBUKFCvzqd4wxpiYi1Rwf8ZA7DoTqEGNMrdvtvvq+3876GUXuOOS8yc7eYIR6Bzn2YP1ms9kbjUabIAhSb4uFYeiNx+PearX6aIzZpF0vf/4UPGZYJyLidTqdL0EQNLIsvnSDXRVqGIZe2i/ebreeiEgURd5yuaz/z/VFfvdisWhl3YP9939i1tpvIpI61DiOM+//JklSiaLoU7PZ/FEqlWza77bWeiLyPBwOv7darZ9p1sZx/GE6nZ7d0vsN7NE8cP+IF6cAAAAASUVORK5CYII="">",0(0.0%)
2,availability_365[int64],Mean (sd) : 205.7 (114.6)min < med < max:0.0 < 193.0 < 365.0IQR (CV) : 232.0 (1.8),366 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/TGe4hAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACg0lEQVR4nO3bTYriQBiH8bfGVqJCgjTozgsMzMKlh5jDzhFm4Qm8gTtBJGKmg/Fj1gN+JtZU/cPz20p11+LBvIkVd7lcDIjdt9AbAJ7xEXoDaA/nXN/MejWXHy6Xy9etDwkVb+Gc639+fv5M03RUZ/1ut9s6537dipVQ8S69NE1H8/n8K8uy8pWFeZ4ni8VitNlsemZGqPAvy7JyPB7/qbG0f+9DbqYggVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhIep3pny+fgst0Ybq+/VbaIk2VPP8+i20xByqmfl7/RZauJmCBEKFBEKFhOhn1FB4NBYXQr2CR2PxIdTreDQWGUK9Q/HRWFtHFkJtkTaPLITaLq0dWQi1hRRHlkd4jgoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJXn9CbXiSJz2fz9137ge6vIXa9CRPVVXJ6XT6XpblbzOr87s1WsTnN2rtkzxmZqvVarRcLgfH47HjY3PQ8jDUBpfv9Hw+d7Ms29Y5ybPdbhud5DmdTl0zS51zdZYHGzsYl667G2qTy3fIS3dRFN2qqn5Mp9NOp9N5+ds81N4Zl2579I1a+/Id8tJ9OBw+kiQZzGazcjKZbF9dH3DvjEs3PDWj1jmI2/TS/Q7D4bDWAeLQe6978Dn0vn3ihL8HDebj1s6YTRHqmzWZj0PPmDHfgBLqmzWZj0POmLHfgBKqJ3Xm45AzZuw3oISKf8R6A8qhFEggVEggVEggVEggVEggVEggVEggVEh46oF/nufJq394v98nZmZFUSTr9XrwP9er/m/VfTdd/0xffwHXXKj5cWeWVAAAAABJRU5ErkJggg=="">",0(0.0%)
3,availability_60[int64],Mean (sd) : 43.9 (19.8)min < med < max:0.0 < 54.0 < 60.0IQR (CV) : 27.0 (2.2),61 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/TGe4hAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAC20lEQVR4nO3Yu07jQBSA4TMJCc5FHpmwpqPZEmmLSDR+gC33YXkEihRbU2yzFR1SFNnCxIpxvMVeRLFAPBMTjvm/NjmTkfJrMo6p61qA96536A0Auzg69AbQHcaYkYgMHcc3dV2vn3uRULEXxpjRbDb7FoZh5DKfZdnKGHP1XKyEin0ZhmEYJUmyttYWTQbTNA0Wi0W0XC6HIkKoaJ+1tojj+MFhdPTSizxMQQVChQqEChUIFSoQKlQgVKhAqFCBUKECoUIFQoUKhAoVCBUqECpUIFSoQKhQgVChAqFCBUKFCoQKFQgVKhAqVCBUqECoUIFQoQKhQgVChQqEChUIFSoQKlQgVKhAqFCBUKECoUIFQoUKhAoVCBUqECpUIFSoQKhQgVChAqFCBUKFCoQKFQgVKhAqVCBUqECoUIFQoQKhQgVChQpHr73BGDMSkaHj+pu6rteOs8A/L4ZqjBnNZrNvYRhGLotnWbYyxlwRqw6eh1K43W4H+9zPU6+dqMMwDKMkSdbW2qLJwmmaBovFIloul0MRIdR3zvdQKssyqKrqoiiKaxF52PP2Xv/pFxGx1hZxHLt8+MhhBofhfCiJiNze3kY3Nzfjx8fHfhub2ylUfByuh9JqtWr1UCLUjvG4Z7Z6x/RFqM/wfLDoicj2ALPBycnJV2vtpOlg23dMX4T6Hz4PFlVVDfI8/zydTn/2er3yrWZFfscmIheXl5ffT09Psyazbd8xfXU2VN+/WqbTaZwkSdb0weLPF/5pPp//ODs7W73V7JP5cRAEZdN7Ztt3TF+dDHVff7UcHx9fu37hk8mk8UOJz+zT+S5qNdSqqgYiEhpjXJdwva85n4gi7/9n8CNqLdQ8zwdlWX45Pz/v9/v9xrH43Nd8TkSRbp9MWrUW6mazOQqCYDyfzwuP+5bPXY8TsUN2CjVN06Dpwvf3941n9i3P8+Du7m7cdO7v3l3mDzWr+bN36esXeL1AfPTZrhYAAAAASUVORK5CYII="">",0(0.0%)
4,bathrooms[float64],Mean (sd) : 1.6 (

# 2. One hot encoding

- Variáveis categóricas com múltiplas categorias com valores faltantes terão uma categoria adicional `nan` adicionada antes do processo de one hot encoding
- Variáveis binárias serão convertidas para valores numéricos (1 e 0)
- A variável `neighbourhood_cleansed` contém muitas categorias pouco frequentes. Agrupamos todas categorias que aparecem em menos de 1% dos dados na categoria `outros`

De 43 colunas inicialmente, terminamos o processo com 63 colunas. 

In [8]:
df_one_hot = df_inicial.copy()

# Calcula a frequência de cada categoria
neighbourhood_counts = df_one_hot['neighbourhood_cleansed'].value_counts(normalize=True)

# Identifica categorias que representam menos de 1%
small_neighbourhoods = neighbourhood_counts[neighbourhood_counts < 0.01].index

# Mapeia essas categorias para 'outros'
df_one_hot['neighbourhood_cleansed'] = df_one_hot['neighbourhood_cleansed'].replace(small_neighbourhoods, 'outros')

# One hot encoding
df_one_hot = pd.get_dummies(df_one_hot, drop_first=True, dummy_na=True)

# Converter colunas booleanas para 1 e 0
boolean_cols = df_one_hot.select_dtypes(include='bool').columns
df_one_hot[boolean_cols] = df_one_hot[boolean_cols].astype(int)

df_one_hot.shape

(26958, 63)

# 4. Imputação de missings

Utilizamos o KNNImputer para imputar missings em variáveis numéricas.
Normalizamos também o nome das colunas.

In [11]:
# KNNImputer
# df_knn = pd.DataFrame(KNNImputer(n_neighbors=5, copy=False).fit_transform(df_one_hot), columns=df_one_hot.columns)

# The Kernel crashed while executing code in the current cell or a previous cell. 
# Please review the code in the cell(s) to identify a possible cause of the failure. 
# Click here for more info. 
# View Jupyter log for further details.

df_knn = pd.read_parquet('../data/bronze/feature_engineering_knn.parquet')
df_knn.columns = df_knn.columns.str.lower().str.replace(' ', '_')
df_knn.to_parquet('../data/bronze/feature_engineering_output.parquet')
df_knn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26958 entries, 0 to 26957
Data columns (total 63 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   accommodates                                     26958 non-null  float64
 1   availability_365                                 26958 non-null  float64
 2   availability_60                                  26958 non-null  float64
 3   bathrooms                                        26958 non-null  float64
 4   bedrooms                                         26958 non-null  float64
 5   beds                                             26958 non-null  float64
 6   calculated_host_listings_count_entire_homes      26958 non-null  float64
 7   calculated_host_listings_count_private_rooms     26958 non-null  float64
 8   calculated_host_listings_count_shared_rooms      26958 non-null  float64
 9   days_since_first_review     

# 4. Conclusão

Aplicamos técnicas como one-hot encoding e imputação de valores ausentes com o KNNImputer, resultando em um conjunto de dados robusto e sem lacunas. Dessa forma estaremos melhor preparados para realizar análises preditivas e extrair insights significativos.